- Notebook réalisé par Aristide LALOUX, Hugo QUENIAT et Mohamed Ali SRIR. 
# Sujet : 
Ce notebook contient le code nécessaire pour créer et entrainer un agent à jouer à Space Invaders sur Atari 2600.
Nous utilisons ici un réseau de neurones convolutionnel (CNN=Convolutional Neuronal Network). C'est à dire que l'agent fait passer l'image en cours du jeu dans une boîte noire qui lui renvoie l'action optimale à jouer. 
Nous y utiliserons donc un algorithme de type DQN (Deep Q Network) qui correspond à du Deep Q Learning.

# Bibliographie :
Nicolas Renotte's code :
- Tutorial : https://www.youtube.com/watch?v=hCeJeq8U0lo
- Git Repository : https://github.com/nicknochnack/KerasRL-OpenAI-Atari-SpaceInvadersv0/actions 
- Agent entrainé avec 1M de steps : https://drive.google.com/file/d/1TgfGittIQC2KhNbut2l4NSASUr0swe1u/edit


# Les installations des bibliotèques

Commençons par installer les différentes librairies que l'on va utiliser.
Il nous faut :
- L'environnement de jeu Space Invaders sur Atari 2600.
- Un agent et de quoi créer un CNN.
- L'algorithme de DQN.

In [ ]:
#Installation des différentes librairies que l'on va utiliser

#1 Gym pour l'environnement du jeu : C'est à dire l'ensemble S des états, l'ensemble A des actions et l'ensemble R des récompenses.
# L'environnement se trouve dans un ROM qui permet d'émuler le jeu sur une machine autre que la console atari.
!{sys.executable} -m pip install autorom[accept-rom-license]
!{sys.executable} -m pip install gym[accept-rom-license]
!{sys.executable} -m pip install gym gym[atari]

#2 Il faut ensuite installer les librairies de Réseaux Neuronaux et d'algorithmes de Deep Q Network (DQN) 
!{sys.executable} -m pip install tensorflow keras-rl2

# Création de l'environnement de jeu et premier essai

- Imports des bibliothèques :

In [3]:
#Importons les bibliothèques et packages qui permettent de créer l'environnement de SpaceInvaders
import gym #La bibliothèque gym contient beaucoup d'environnements déjà codés et notamment SpaceInvaders
import random #La bibliothèque random nous permettra de choisir des actions aléatoires ou bien d'implémenter l'epsilon-greedy policy

- Création de l'environnement de jeu Space Invaders Atari 2600:


Les états correspondent à toutes les possibilités de position de notre vaisseau et des vaisseaux ennemis sur l'écran. Cela fait beaucoup d'états possibles, c'est pourquoi on utilise le DQN plutôt qu'un QL basique.
Il y a 6 actions : 

{Ne_rien_faire / Tirer / Déplacement_Droite / Délacement_Gauche / Déplacement_Droite & Tirer / Déplacement_Gauche & Tirer}.

Les récompenses correspondent simplement au score dans le jeu. Celui-ci augmente à chaque vaisseau ennemi détruit, la récompense obtenue à chaque vaisseau éliminé étant proportinnelle à la distance entre le joueur et le vaisseau.


In [13]:
#On importe l'environnement Gym SpaceInvaders que l'on retrouve directement sur leur site internet. 
env = gym.make('SpaceInvaders-v0')

#On récupère la taille des images que l'environnement génère. 
height, width, channels = env.observation_space.shape
print(env.observation_space.shape) 

#On récupère les actions permises à la futur IA. (L'ensemble A)
env.unwrapped.get_action_meanings()
print(env.unwrapped.get_action_meanings())
actions = env.action_space.n
print(env.action_space.n)
#Ici on voit qu'il y a 6 actions contenues dans une liste. L'action 1 correspond à FIRE (donc tiré), 4 à LEFT (donc déplacer notre
#vaisseau d'une unité à gauche)

(210, 160, 3)
['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']
6


C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gym\envs\registration.py:564: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Prenons le temps de faire jouer un agent de manière totalement aléatoire. Premièrement, cela permet de se familiariser à la syntaxe proposée par Gym. Ensuite, il sera intéressant de comparer les résultats d'une tentative de jeu totalement aléatoire avec une tentative de jeu d'une IA entrainée. On espère évidemment que notre agent entrainé fasse mieux qu'une tactique de jeu aléatoire !

In [12]:
episodes = 5 #On va lancer 5 épisodes = tentatives de jeu.
for episode in range(1, episodes+1):
    state = env.reset() #On reset la partie à chaque début de partie, logique!
    done = False #Done vaut false tant que l'agent n'est pas mort ou n'a pas gagné le jeu. Done indique ainsi une fin de partie.
    score = 0 #Le score vaut la somme des rewards obtenus au fur et à mesure des itérations. 
    
    while not done: #Tant que le jeu n'est pas fini : IE l'agent n'est pas mort ou n'a pas éliminé tous les ennemis.
        env.render(mode="rgb_array") #Pour afficher la fenêtre de jeu. 
        action = random.choice([0,1,2,3,4,5]) #On prend ici des actions totalement aléatoires.
        n_state, reward, done, info = env.step(action) #La fonction step de gym s'occupe d'appliquer l'action "action" à l'état actuel pour 
        #calculer le prochain état. 
        score+=reward #On stocke le cumul des rewards dans "score"
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gym\utils\passive_env_checker.py:305: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps
  logger.warn(


Episode:1 Score:65.0
Episode:2 Score:205.0
Episode:3 Score:140.0
Episode:4 Score:120.0
Episode:5 Score:110.0


# Création du CNN (Convolutionnal Neuronal Network) avec Keras

- Nous allons ici implémenter notre réseau neuronal. Il faut que l'IA soit capable de prendre une image en entrée et de ressortir la meilleur action à jouer. 

In [22]:
import numpy as np

from tensorflow.keras.models import Sequential #Bibliothèque pour créer un CNN de manière "séquential", c'est à dire qu'on va lui ajouter de manière séquentielle des couches une à une. 
from tensorflow.keras.layers import Dense, Flatten, Convolution2D #Bibliothèques des couches de CNN.
from tensorflow.keras.optimizers import Adam #Algo pour optimiser l'entrainement.

C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
C:\Users\lalou\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
C:\Users\lalou\AppData\Local\Packages\

- On écrit la fonction qui permet d'implémenter le réseau neuronal

In [23]:
def build_model(height, width, channels, actions): #On lui passe en argument la taille des images en entrée et la taille de la liste des actions.
    
    model = Sequential() #On déclare que noous allons construire le modèle de façon séquentielle, ie on va lui ajouter successivement des couches. 

    #On commence par des couches de convolution pour traiter les images.
    
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    #Ici : 32 correspond au nombre de filtres, (8,8) correspond à la taille de la matrice du filtre, strides=(4,4) correspond au pas de placement
    #du filtre entre deux calculs et activation='relu' correspond à la fonction d'activatin de couche qui est ici une relu.

    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    #Idem
    model.add(Convolution2D(64, (3,3), activation='relu'))
    #Idem. Attention ici strides n'est pas précisé, par défault il vaut 1 et donc le filtre se déplace de pixel en pixel et passe donc
    #par tous les pixels.

    model.add(Flatten())
    #Cette couche permet de reprendre toutes les imagettes calculées en sortie des 32*64*64 filtres et de tout mettre dans un grand tableau. 

    model.add(Dense(512, activation='relu'))
    #Il s'agit ici d'une couche fully-connected de taille 512 en sortie avec une fonction d'activation de type "relu"
    model.add(Dense(256, activation='relu'))
    #Il s'agit ici d'une couche fully-connected de taille 256 en sortie avec une fonction d'activation de type "relu"
    model.add(Dense(actions, activation='linear'))
    #Il s'agit ici d'une couche fully-connected de taille actions/6 en sortie avec une fonction d'activation de type "linéaire"
    return model

- Nous créons à présent le réseau neuronal. Il sera stocké dans la variable model.

In [24]:
model = build_model(height, width, channels, actions)

- Nous pouvons obtenir des informations sur les couches de notre réseau : input, output, dimensions, nombre de variable, etc...

In [ ]:
model.summary()
#Pour le calcul des dimensions à chaque couche https://www.baeldung.com/cs/convolutional-layer-size

#Pour une couche convolutive, avec un filtre, on a les formules suivantes:
#Height_output = (Height_input - Height_filter)/(HeightStridesFilter) +1
#width_output = (Witdh_input - Witdh_filter)/(WitdhStridesFilter) +1

#On retrouve bien les valeurs que le tableau contient. 
#Remarque : le "3" étant pour le RGB des pixels (Red, Green, Blue)

# Création de l'agent à l'aide de Keras.RL

- Nous entrons à présent dans la partie qui nous intéresse particulièrement : nous allons entrainer notre agent pour jouer à Space Invaders.
- Commençons par importer les librairies python.

In [11]:
#Nous allons utiliser la bibliothèque DQNAgent de Keras RL qui facilite la création de notre Agent pour l'algorithme de DQN
from rl.agents import DQNAgent


#Notre agent à besoin d'un buffer/mémoire pour y stocker les informations qu'il a appris lors de son entrainement.
from rl.memory import SequentialMemory


#Enfin les politiques epsilon greedy et epsilon greedy décroissante existent déjà dans les bibliothèques ci dessous.
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
#EpsGreedyQPolicy correspond à une politique EpsilonGreedy où Epsilon est constant
#LinearAnnealedPolicy correspond à une politique EpsilonGreedy où Epsilon décroit linéairement.

- Créons donc la fonction d'initialisation de notre Agent.

In [12]:
def build_agent(model, actions):
    #Nous devons définir la politique de l'agent. Nous choisissons une politique Epsilon-greedy qui décroit avec le temps.
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    #attr="eps" indique que nous allons donner nos spécifications pour epsilon
    #value_max=1. indique que la valeur intiale de epsilon est 1
    #value_min=.1 indique que la valeur finale de epsilon est 0.1
    #value_test=.2 est plus subtil. A chaque étape, on tire un nombre entre 0 et 1 pour savoir si on choisit l'action qui maximise Q ou si on choisit d'explorer. Si ce nombre tiré 
    #est plus petit que value_test, alors on décide de diminuer epsilon vers 0.1
    #nb_steps=10000 indique le pas de descente de epsilon. Il faut 10000 étapes où le nombre est plus petit que value_test pour que epsilon passe de value_max à value_min


    #Nous devons créer le buffer dans lequel notre agent mémorise les informations de son entrainement
    memory = SequentialMemory(limit=1000, window_length=3)
    #limit=1000 indique que notre buffer contient au maximum 1000 épisodes/ tentatives de jeu 
    #window_length=3 indique que nous utilisons 3 fenêtres de buffer pour les 1000 épisodes

    #Créons donc notre agent
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn
    #model: on donne à l'agent le réseau neuronal
    #memory : on donne à l'agent le buffer de mémoire
    #policy : on donne à l'agent la politique à suivre

    #enable_dueling_network permet d'optimiser et d'accélérer le training de l'agent
    #dueling_type permet d'optimiser et d'accélérer le training de l'agent

    #nb_actions=actions : indique à l'agent le nombre d'acions -> ici 6
    #nb_steps_warmup: On laisse à l'agent un warmup/Echauffement de 1000 épisodes avant de commencer à effectivement entrainer l'agent dans les règles de l'art .


- On initialise notre agent

In [ ]:
dqn = build_agent(model, actions)

#On utilise le processus Adam qui permet d'optimiser l'agent au départ et donc de gagner du temps. 
dqn.compile(Adam(lr=1e-4))
#lr correspond au learning rate.

- C'est bon tout est prêt ! C'est parti, entrainons cet agent !

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)
#Cette fonction permet d'entrainer l'agent sur l'environnement "env" pendant "nb_steps".
#Attention il ne faut pas confondre un épisode qui correspond à une tentative de jeu/ un partie entière avec un "step" qui correspond à une unique décision prise par l'agent face à une image.
#Il y a donc plusieurs steps par épisode. (sauf si l'agent meurt dès la première image)


- Voilà nous avons ainsi entrainé notre agent
- Maintenant, il faudrait pouvoir le tester afin de comparer par la suite à d'autres politiques et voir si notre agent devient effectivement de plus en plus fort.
Lançons donc 10 épisodes avec l'option visualize=True pour pouvoir afficher à l'écran l'agent qui joue au jeu.

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))


# Sauvegarder et réutiliser un agent depuis la mémoire

- Voilà, nous avons réussi à entrainer notre agent sur l'environnement SpaceInvaders. Pour qu'il devienne effectivement bon au jeu, il est capital de l'entrainer un très grand nombre de fois.
Nous avons pas forcément le temps ni les capacités computationnelles de le faire. C'est pourquoi il est pertinent de pouvoir enregistrer notre optimisation et pouvoir la réutiliser ultérieurement. (Ne serait ce
pour éviter de tout perdre lors d'un crash de la machine qui travaille, ou encore pour pouvoir importer un agent d'une autre personne).

In [2]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')   
#Notre agent est sauvegardé dans "./SavedWeights/10k-Fast/dqn_weights.h5f"

NameError: name 'dqn' is not defined

- Ci dessous afin de reset notre model et notre agent.

In [ ]:
del model, dqn

- Avant d'éxécuter la cellule ci dessous. Il faut re-compiler les parties 3 et 4 SAUF LA CELLULE dqn.fit qui entraîne l'agent. En effet nous allons importer un agent déjà entrainer
à 1 million de Steps. 

In [ ]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')
#Compiler alors cette cellule puis exécuter la cellule qui teste l'agent pendant 10 épisodes.
#On peut alors comparer les scores de l'agent entrainer 1 million de fois avec l'agent entrainer 10000 fois et celui du début qui prend toutes les décisions aléatoirement.


# Conclusion

En comparant un agent entrainé à 1M de steps avec un agent qui joue aléatoirement, on observe une net différence de niveau de jeu. L'agent entrainé obtient bel est bien des meilleurs scores, ouf !
Cependant on remarque qu'avec 1M de steps de vécu, notre agent entrainé reste pas très bon par rapport à un humain. Il faudrait donc continuer le training... cela prend du temps.